## 정리노트 #2

In [83]:
'''단위 테스트 - 실제로 함수가 잘 동작하는지 확인하는 테스트 / 테스트 대상 단위의 크기를 작게 설정해서 단위 테스트를 최대한 간단하게 작성!'''
''' +) 통합 테스트는 여러 모듈을 모아 개발된 의도대로 동작되는지 확인하는 테스트이다.'''

import unittest #unittest를 import하고 

class SquareTest(unittest.TestCase): #unittest.TestCase를 상속한 SquareTest 클래스를 구현
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expect = np.array(4.0) # 입력이 2.0일 때 출력이 4.0
        self.assertEqual(y.data, expected) # 주어진 두 객체가 동일한지 여부를 판정


터미널에서 *python -m unittest 테스트 코드 파일 주소* 실행 -> -m unittest 인수를 제공하면 파이썬 파일을 테스트 모드로 실행 가능

In [84]:
'''square 함수의 역전파 테스트'''
class SquareTest(unittest.TestCase): 
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expect = np.array(4.0) 
        self.assertEqual(y.data, expected) 
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward() #미분값을 구함
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [85]:
'''기울기 확인을 이용한 자동 테스트'''
# 수치 미분으로 구한 결과와 역전파로 구한 결과를 비교하여 그 차이가 크면 역전파 구현에 문제가 있다고 판단하는 검증 기법

#수치 미분 구현 코드
def numerical_diff(f,x,eps=1e-4): # eps는 미소한 변화량을 의미하며,x의 값을 조금씩 변화시키기 위해 사용
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expect = np.array(4.0) 
        self.assertEqual(y.data, expected) 
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward() 
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
    # 기울기 확인
    def test_gradiant_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

두 메서드로 구한 값들이 일치하는지 확인하기 위해 np.allclose라는 넘파이 함수를 사용한다. 
np.allclose(a,b)는 ndarray 인스턴스인 a와 b의 값이 가까운지 판정한다.  

In [86]:
'''가변 길이 인수(순전파)'''

'''원래의  Function 코드'''
class Function:
    def __call__(self, input):
        x = input.data # Variable이라는 상자에서 실제 데이터를 꺼낸 다음
        y = self.forward(x) #forward 메서드에서 구체적인 계산.
        output = Variable(y) # 계산 결과를 Variable에 넣고 
        output.set_creator(self) # 자신이 창조자라고 설정
        self.input = input
        self.output = output 
        return output
    
def forward(self, x):
    raise NotImplementedError()
    
def backward(self, gy):
    raise NotImplementedError()

가변 길이란 인수 또는 반환값의 수가 달라질 수 있다는 것을 말한다.

In [87]:
'''수정된  Function 코드'''

class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # 변수를 리스트에 담아 취급한다
        ys = self.forward(xs) 
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)        
        self.inputs = inputs
        self.outputs = outputs 
        return outputs
    
def forward(self, xs):
    raise NotImplementedError()
    
def backward(self, gys):
    raise NotImplementedError()

**<리스트 내포> = [표현식 for 항목 in 순회 가능 객체 if 조건]**

순회 가능한 어떤 객체에서 새로운 리스트를 생성할 수 있으며, 필요한 경우 각 요소에 대해 특정 조건을 적용할 수도 있다.
여기서 "표현식"은 새 리스트의 요소에 대한 계산이나 조작을 정의하며, "조건"은 해당 표현식이 적용될 요소를 필터링하는 데 사용된다.


+) x0, x1 = xs 

파이썬의 **언패킹(unpacking) 기능**은 컬렉션(예: 리스트, 튜플, 딕셔너리 등)의 요소를 여러 변수에 한 번에 할당할 수 있게 해 준다.

In [88]:
'''덧셈을 해 주는 Add 클래스를 구현, 인수와 반환값이 리스트 또는 튜플이어야 한다'''

class Add(Function):
    def forward(self, xs):
        x0, x1 = xs #언패킹
        y = x0 + x1
        return(y, ) #단일 요소를 포함하는 튜플을 반환

위 코드의 개선점 : 입력시 변수를 리스트로 전달하도록 요청하고 반환값도 튜플로 전달하므로 사용 시 복잡함

In [89]:
'''개선된  Function 코드'''

class Function:
    def __call__(self, *inputs): # *를 붙인다
        xs = [x.data for x in inputs] 
        ys = self.forward(xs) 
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)        
        self.inputs = inputs
        self.outputs = outputs 
        return outputs
        # 리스트의 원소가 하나라면 첫 번째 원소를 반환
        return outputs if len(outputs) > 1 else outputs[0]
    
def forward(self, xs):
    raise NotImplementedError()
    
def backward(self, gys):
    raise NotImplementedError()

함수를 정의할 때 인수에 *를 붙이면 호출할 때 넘긴 인수들을 *를 붙인 인수 하나로 모아서 받을 수 있다!

In [90]:
def as_array(x):
    if np.isscalar(x):  
        return np.array(x)  # 해당 경우 배열로 변환
    return x

In [91]:
'''입력과 결과를 반환할 때 변수를 직접 사용할 수 있도록 다시 개선된 코드'''

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) # *를 붙여 언팩 / 리스트의 원소를 낱개로 풀어 전달
        if not isinstance(ys, tuple): # 튜플이 아닐 경우 튜플로 변경한다.
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)        
        self.inputs = inputs
        self.outputs = outputs 
    
        # 리스트의 원소가 하나라면 첫 번째 원소를 반환
        return outputs if len(outputs) > 1 else outputs[0]

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1 
        return y

원래 ADD 클래스는 x0, x1 = xs처럼 언패킹을 해 주고 y = x0 + x1 계산을 해야 했었는데, 개선된 코드는 함수를 호출할 때 *를 붙여 리스트 언팩을 했으므로 변수를 직접 받을 수 있다. 또 튜플이 아닐 경우 튜플로 변경되므로 return(y, )가 아니라 return y로 원소 하나만 반환하는 
코드 개선이 가능하다.

In [92]:
# ADD 클래스를 파이썬 함수로 사용할 수 있는 코드 추가
def add(x0, x1):
    return Add()(x0, x1)

In [93]:
# 실행을 위해 note#1에 구현한 Variable 클래스 코드를 가져옴 
import numpy as np

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) #np.ones_like(self.data)는 self.data와 형상과 타입이 같은 ndarray인스턴스 생성.
                                                #모든 요소를 1로 채워서 돌려줌
            
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [94]:
# add()를 이용하여 계산 코드를 작성

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) # Add 클래스 생성 과정이 감춰짐 / 개선 전에는 f = Add()가 필요!
print(y.data)

5


In [95]:
'''가변 길이 인수(역전파)'''

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self,gy): # 입력이 1개
        return gy, gy # 출력이 2개

덧셈의 역전파는 출력 쪽에서 전해지는 미분값에 1을 곱한 값이 입력 변수(x0, x1)의 미분이다. 

In [96]:
'''Variable 클래스 수정'''
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = [self.creator] # funcs 리스트에 함수를 추가함으로써 역전파가 진행될 경로를 설정하는 것이다!
        while funcs: # funcs 리스트(스택)에 처리할 함수가 남아있는 동안, 반복문을 실행
            f = funcs.pop() #  마지막 함수를 꺼내서 f에 저장
            gys = [output.grad for output in f.outputs] # 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담는다
            gxs = f.backward(*gys) # 함수 f의 역전파를 호출한다 + 리스트 언팩을 통해 리스트를 풀어준다
            if not isinstance (gxs, tuple): # gxs가 튜플이 아니라면 튜플로 변환
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분값을 grad에 저장
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)


zip 함수는 zip(리스트1, 리스트2) 를 한개로 처리하여 리스트의1 의 요소A, 리스트2의 요소 B를 반환

다변수 함수들을 행렬로 바꿔서 값을 하나씩 미분하고 가중치를 업데이트 한 뒤 zip()으로 묶어줌

In [97]:
'''Square 클래스 수정'''

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data # 변수 이름이 단수인 input에서 복수인 inputs으로 바뀜
        gx = 2 * x * gy
        return gx 

In [98]:
# 실행을 위해 note#1에 구현한 코드를 가져옴
'''파이썬 함수로 이용하기'''
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [99]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
z = add(square(x), square(y))

z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [100]:
'''같은 변수 사용 시 발생되는 문제점을 해결하기 위해 처음 이후 부터 전달된 미분값을 더해주도록 수정해야 함'''

x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)

y.backward()
print('x.grad', x.grad) # -> 잘못된 값이 나옴(미분값이 덮어 써지기 때문이다)

y 6.0
x.grad 1.0


In [101]:
'''Variable 클래스 수정'''
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None: # 미분값을 처음 설정하는 경우는 출력 쪽에서 전해지는 미분값을 그대로 대입
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 다음번부터는 전달된 미분값을 더해줌
                
                if x.creator is not None:
                    funcs.append(x.creator)


In [102]:
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 2.0


In [103]:
'''여러 가지 미분을 연달아 계산할 때 똑같은 변수를 재사용 할 수 있도록 클래스에 미분값을 초기화하는 cleargrad 메서드를 추가해 준다'''

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
    def cleargrad(self):
        self.grad = None

In [104]:
#첫 번째 계산
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)
#두 번째 계산
x.cleargrad() # 미분값 초기화
y = add(add(x,x), x)
y.backward()
print(x.grad) # 두 번째 x.backward()를 호출하기 전에 x.cleargrad()를 호출하면 변수에 누적된 미분값이 초기화 됨!

2.0
3.0


지금까지 구현한 계산 그래프는 한줄로 나열되어 리스트에서 원소(함수)를 꺼내는 순서를 고려하지 않아도 괜찮았지만, 다양한 위상의 계산 그래프에 대응하기 위해선 함수에 **우선순위**를 줄 수 있어야 한다. 

우선순위를 설정하기 위해 함수와 변수의 세대를 기록하여 역전파 시 세대 수가 큰 쪽부터 처리하면 올바른 순서로 진행할 수 있다.

In [105]:
'''세대 추가'''

class Variable:
    def __init__(self,data):
        if data is not None:
            if not ininstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록한다(부모 세대 + 1)
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
   
    def cleargrad(self):
        self.grad = None

In [106]:
'''Function 클래스의 generation은 입력 변수와 같은 값으로 설정한다. 입력 변수가 둘 이상이라면 가장 큰 generation의 수를 선택한다.'''

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs]) # Function의 generation 설정
        for output in outputs:
            output.set_creator(self)        
        self.inputs = inputs
        self.outputs = outputs 
    
        return outputs if len(outputs) > 1 else outputs[0]

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1 
        return y

**key = lambda x: x.generation**

리스트의 원소를 x라고 했을 때 x.generation을 키로 사용해 정렬하라

In [107]:
'''Variable 클래스의 backward 메서드 구현 - add_func 함수 추가'''

class Variable:
    def __init__(self,data):
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = []
        seen_set = set()
        
        def add_func(f): # 역전파를 수행할 때 사용되는 함수 객체들을 관리하는 역할 / 리스트를 세대 순으로 정렬 
            if f not in seen_set: # 같은 함수를 중복해서 처리하는 것을 방지
                funcs.append(f) # 처리해야 할 함수들을 funcs 리스트에 추가
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() # 세대 순으로 정렬되었기 때문에 자동으로 세대가 가장 큰 함수를 꺼낸다
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator) # 수정 전 : funcs.append(x.creator)
   
    def cleargrad(self):
        self.grad = None

add_func 함수를 backward 메서드 안에 **중첩 함수**로 정의하였다. 중첩 함수는 다음 두 조건을 충족할 때 적합하다.

1. 감싸는 메서드 안에서만 이용한다.
2. 감싸는 메서드에 정의된 변수를 사용해야 한다.

In [113]:
# 동작 확인을 위해 위에 정의한 함수를 다시 가져옴

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data 
        gx = 2 * x * gy
        return gx 
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self,gy): 
        return gy, gy 
    
def square(x):
    f = Square()
    return f(x)

In [114]:
'''동작 확인'''

x = Variable(np.array(2.0))
a = square(x)
y = add(square(a),square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
